In [ ]:
cd /content/drive/MyDrive/uniOULU/AffectiveComputing/project

/content/drive/MyDrive/uniOULU/AffectiveComputing/project


In [ ]:
from transformers import AutoTokenizer, AutoModel

# 定义模型和分词器名称
model_name = "FacebookAI/xlm-roberta-base"

# 下载模型和分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 保存到本地目录
local_model_dir = "./xlm-roberta-base"  # 你可以自定义路径
tokenizer.save_pretrained(local_model_dir)
model.save_pretrained(local_model_dir)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
!zip -r /content/xlm-roberta-base.zip /content/xlm-roberta-base

  adding: content/xlm-roberta-base/ (stored 0%)
  adding: content/xlm-roberta-base/tokenizer_config.json (deflated 76%)
  adding: content/xlm-roberta-base/sentencepiece.bpe.model (deflated 49%)
  adding: content/xlm-roberta-base/special_tokens_map.json (deflated 52%)
  adding: content/xlm-roberta-base/tokenizer.json (deflated 76%)
  adding: content/xlm-roberta-base/config.json (deflated 50%)
  adding: content/xlm-roberta-base/model.safetensors (deflated 42%)


In [ ]:
!cp /content/xlm-roberta-base.zip /content/drive/MyDrive/uniOULU/AffectiveComputing/project/

reddit爬数据

In [ ]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# !! use your own credentials !!
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

import praw
import pandas as pd
import re
from datetime import datetime, timezone, timedelta

# ==== Reddit credentials ====
reddit = praw.Reddit(
    client_id="xxxxxxx",
    client_secret="x-x",
    user_agent="x-x-x",
    username="xxx",
    password="xxx",
    check_for_async=False
)

# ==== Time range (UTC) ====
# Define start and end dates dynamically for each day between 9/1 and 10/12
start_date = datetime(2025, 9, 16, 0, 0, 0, tzinfo=timezone.utc)
end_date = datetime(2025, 10, 13, 0, 0, 0, tzinfo=timezone.utc)
days = pd.date_range(start=start_date, end=end_date, freq='D')

# ==== Scraping parameters ====
subreddit_name = "Trump"
limit_posts = 1000  # Maximum number of posts to check
limit_comments_per_post = 40  # Maximum comments per post
daily_limit = 200  # Number of comments per day

# ==== Begin scraping ====
subreddit = reddit.subreddit(subreddit_name)

data = []  # List to store scraped data
count = 0  # Total comment count across all days

print(f"📅 Collecting comments from r/{subreddit_name} from {start_date.date()} to {end_date.date()}...")

# Loop through each day in the date range
for day in days:
    # Calculate the start and end timestamp for the current day
    day_start = day
    day_end = day + timedelta(days=1)
    start_timestamp = day_start.timestamp()
    end_timestamp = day_end.timestamp()

    day_count = 0  # Number of comments collected for this day

    print(f"📅 Collecting comments for {day.date()}...")

    # Loop through posts for the current day
    for submission in subreddit.new(limit=limit_posts):
        if day_count >= daily_limit:
            break  # Stop when we've collected enough comments for the day

        # Check if post is within the current day time window
        if start_timestamp <= submission.created_utc <= end_timestamp:
            submission.comments.replace_more(limit=0)

            # Iterate through the comments of the submission
            for comment in submission.comments[:limit_comments_per_post]:
                if day_count >= daily_limit:
                    break  # Stop if we've reached the daily comment limit

                text = comment.body.strip()

                # Only collect non-deleted, non-removed comments
                if text and text.lower() not in ["[deleted]", "[removed]"]:
                    data.append({
                        "post_id": submission.id,
                        "comment_id": comment.id,
                        "post_title": submission.title,
                        "created_utc": datetime.fromtimestamp(comment.created_utc, tz=timezone.utc),
                        "score": comment.score,
                        "text": text
                    })
                    day_count += 1
                    count += 1

    print(f"✅ Collected {day_count} comments for {day.date()}.")

# If we collected any comments, save them
if data:
    df = pd.DataFrame(data)
    filename_full = f"{subreddit_name}_{start_date.date()}_{end_date.date()}_{int(datetime.now().timestamp())}.csv"
    df.to_csv(filename_full, index=False, encoding="utf-8-sig")
    print(f"💾 Full dataset saved: {filename_full}")

    # ==== Clean text and export text-only CSV ====
    def clean_text(text):
        # Remove URLs, markdown links, mentions, and excessive whitespace
        text = re.sub(r"http\S+|www\.\S+|discord\.gg/\S+", "", text)  # Remove URLs
        text = re.sub(r"\[.*?\]\(.*?\)", "", text)  # Remove markdown links
        text = re.sub(r"\*{1,2}|\~|\_|\>", "", text)  # Remove markdown symbols
        text = re.sub(r"\s+", " ", text).strip()  # Clean excessive whitespace

        # Remove bot-related text
        if "I am a bot" in text or "contact the moderators" in text:
            return ""

        return text

    # Clean and save text-only CSV
    text_only = df["text"].dropna().apply(clean_text)
    text_only = text_only[text_only != ""].reset_index(drop=True)

    filename_text = f"{subreddit_name}_{start_date.date()}_{end_date.date()}_text_{int(datetime.now().timestamp())}.csv"
    text_only.to_csv(filename_text, index=False, header=["text"], encoding="utf-8-sig")
    print(f"💾 Clean text-only file saved: {filename_text}")
else:
    print("⚠️ No comments found — text-only file not generated.")

print("✅ All tasks completed successfully.")


📅 Collecting comments from r/Trump from 2025-09-16 to 2025-10-13...
📅 Collecting comments for 2025-09-16...
✅ Collected 200 comments for 2025-09-16.
📅 Collecting comments for 2025-09-17...
✅ Collected 200 comments for 2025-09-17.
📅 Collecting comments for 2025-09-18...
✅ Collected 200 comments for 2025-09-18.
📅 Collecting comments for 2025-09-19...
✅ Collected 200 comments for 2025-09-19.
📅 Collecting comments for 2025-09-20...
✅ Collected 200 comments for 2025-09-20.
📅 Collecting comments for 2025-09-21...
✅ Collected 200 comments for 2025-09-21.
📅 Collecting comments for 2025-09-22...
✅ Collected 200 comments for 2025-09-22.
📅 Collecting comments for 2025-09-23...
✅ Collected 180 comments for 2025-09-23.
📅 Collecting comments for 2025-09-24...
✅ Collected 200 comments for 2025-09-24.
📅 Collecting comments for 2025-09-25...
✅ Collected 200 comments for 2025-09-25.
📅 Collecting comments for 2025-09-26...
✅ Collected 200 comments for 2025-09-26.
📅 Collecting comments for 2025-09-27...
✅

In [ ]:
!rm -r *.csv

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 12.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import emoji

# Function to clean the text
def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\.\S+|discord\.gg/\S+", "", text)

    # Remove markdown links
    text = re.sub(r"\[.*?\]\(.*?\)", "", text)

    # Remove markdown symbols like * ** ~ _ >
    text = re.sub(r"\*{1,2}|\~|\_|\>", "", text)

    # Replace emojis with their textual representation (e.g., 😊 -> happy_face)
    text = emoji.demojize(text, delimiters=("", ""))

    # Remove extra whitespaces and strip the text
    text = re.sub(r"\s+", " ", text).strip()

    return text

# Load the CSV file
df = pd.read_csv("/content/Trump_2025-09-16_2025-10-13_1760423874.csv")

# Step 1: Keep only 'created_utc' and 'text'
df = df[['created_utc', 'text']]

# Step 2: Sort by 'created_utc' in ascending order (oldest first)
df['created_utc'] = pd.to_datetime(df['created_utc'])  # Ensure datetime format
df = df.sort_values(by='created_utc', ascending=True)

# Step 3: Clean the text (URL removal, emoji conversion, etc.)
df['text'] = df['text'].apply(clean_text)

# Step 4: Truncate the text to a maximum of 512 characters
df['text'] = df['text'].apply(lambda x: x[:512] if len(x) > 512 else x)

# Save the processed CSV
df.to_csv("processed_data.csv", index=False, encoding="utf-8-sig")

print("✅ Data processing completed. Saved to 'processed_data.csv'")


✅ Data processing completed. Saved to 'processed_data.csv'


In [ ]:
import pandas as pd

# 读取CSV文件
input_csv = "processed_data.csv"  # 你的输入文件路径
output_csv = "processed_data_no_empty_text.csv"  # 输出的去除空text后的CSV文件路径

# 读取CSV文件到DataFrame
df = pd.read_csv(input_csv)

# 删除text为空或NaN的行
df_cleaned = df.dropna(subset=['text'])  # 删除text列为NaN的行
df_cleaned = df_cleaned[df_cleaned['text'].str.strip() != ""]  # 删除text列为空字符串的行

# 保存清理后的数据
df_cleaned.to_csv(output_csv, index=False, encoding="utf-8-sig")

print(f"✅ 去除空text条目后的数据已保存到 '{output_csv}'")
